# Creating function for sudoku data

In [1]:
# importing required libraries

import csv
import os
import urllib.request
import zipfile
import numpy as np
import pandas as pd

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.dataset import Dataset

import torch
import dgl
from copy import copy

Using backend: pytorch


### Let's understand how the basic sudoku graph is built

for this we will take an example grid of 9*9 and iterate over this to list out all the edges and nodes and convert it into graph like data


In [32]:
grids = [[0, 1, 2, 9, 10, 11, 18, 19, 20],
         [3, 4, 5, 12, 13, 14, 21, 22, 23],
         [6, 7, 8, 15, 16, 17, 24, 25, 26],
         [27, 28, 29, 36, 37, 38, 45, 46, 47],
         [30, 31, 32, 39, 40, 41, 48, 49, 50],
         [33, 34, 35, 42, 43, 44, 51, 52, 53],
         [54, 55, 56, 63, 64, 65, 72, 73, 74],
         [57, 58, 59, 66, 67, 68, 75, 76, 77],
         [60, 61, 62, 69, 70, 71, 78, 79, 80]]

In [33]:
edges = set()

In [34]:
for i in range(81):
        row, column = i // 9, i % 9

        print('for i = ',i)
        print('row',row)
        print('column',column)
        print('----')

        # same row and column
        row_src = row * 9
        print('row source',row_src)
        column_src = column
        print('column source',column_src)


        for j in range(9):

            print('value of j',j)

            # adding edges
            edges.add((row_src,i))
            edges.add((column_src,i))
            #print('these are edges',edges)

            row_src += 1
            column_src += 9
        # same grid

        grid_row, grid_column = row//3, column//3
        print('grid row',grid_row)
        print('grid column',grid_column)

        for n in grids[grid_row*3 + grid_column]:
            if n!= i:
                edges.add((n,i))
                #print('edges again', edges)
edges = list(edges)

for i =  0
row 0
column 0
----
row source 0
column source 0
value of j 0
value of j 1
value of j 2
value of j 3
value of j 4
value of j 5
value of j 6
value of j 7
value of j 8
grid row 0
grid column 0
for i =  1
row 0
column 1
----
row source 0
column source 1
value of j 0
value of j 1
value of j 2
value of j 3
value of j 4
value of j 5
value of j 6
value of j 7
value of j 8
grid row 0
grid column 0
for i =  2
row 0
column 2
----
row source 0
column source 2
value of j 0
value of j 1
value of j 2
value of j 3
value of j 4
value of j 5
value of j 6
value of j 7
value of j 8
grid row 0
grid column 0
for i =  3
row 0
column 3
----
row source 0
column source 3
value of j 0
value of j 1
value of j 2
value of j 3
value of j 4
value of j 5
value of j 6
value of j 7
value of j 8
grid row 0
grid column 1
for i =  4
row 0
column 4
----
row source 0
column source 4
value of j 0
value of j 1
value of j 2
value of j 3
value of j 4
value of j 5
value of j 6
value of j 7
value of j 8
grid row 0
grid

In [35]:
# defining basic sudoku graph


def basic_sudoku_graph():
    grids = [[0, 1, 2, 9, 10, 11, 18, 19, 20],
             [3, 4, 5, 12, 13, 14, 21, 22, 23],
             [6, 7, 8, 15, 16, 17, 24, 25, 26],
             [27, 28, 29, 36, 37, 38, 45, 46, 47],
             [30, 31, 32, 39, 40, 41, 48, 49, 50],
             [33, 34, 35, 42, 43, 44, 51, 52, 53],
             [54, 55, 56, 63, 64, 65, 72, 73, 74],
             [57, 58, 59, 66, 67, 68, 75, 76, 77],
             [60, 61, 62, 69, 70, 71, 78, 79, 80]]
    
    edges = set()

    # now looping through all the 81 nodes 

    for i in range(81):
        row, column = i // 9, i % 9

        # same row and column
        row_src = row * 9
        column_src = column

        for j in range(9):

            # adding edges
            edges.add((row_src,i))
            edges.add((column_src,i))

            row_src += 1
            column_src += 9
        # same grid

        grid_row, grid_column = row//3, column//3

        for n in grids[grid_row*3 + grid_column]:
            if n!= i:
                edges.add((n,i))
    edges = list(edges)

    g = dgl.graph(edges)

    return g


In [36]:
g = basic_sudoku_graph()

In [37]:
g.edges()

(tensor([26, 67, 59,  ..., 76, 15,  7]),
 tensor([21, 59, 55,  ..., 22, 12,  8]))

In [45]:
# Loading the data and examin

df = pd.read_csv(r'C:\Users\ANIMESH\Desktop\ANI PC\PhD Prepratery Work\GNN-Problems\sudoku-hard\sudoku-hard\train.csv')
df.head()

,003000070000900400000000000000037002407000600090006700100000830000050007080200000,923814576756923481814765329568437192437192658291586743172649835649358217385271964
0,0507200000600000900080000004000308000000050609...,1597284367643512982386941574761398258234759619...
1,0152000000000000830000000008000100400005001000...,4152387966291574837834692518763129459325741685...
2,1090304000000000070025000100508073000800009400...,1697324585486192377325846199518473626871239453...
3,0000751032030408000000000004073900001000007450...,9468751232139468577851234694573916821396827458...
4,6000001022000897430005010000048700600050060100...,6893471522516897433475218961348752698752963149...


In [48]:
def read_csv(fname):
        print("Reading %s..." % fname)
        with open(fname) as f:
            reader = csv.reader(f, delimiter=',')
            return [(q, a) for q, a in reader]

data = read_csv(r'C:\Users\ANIMESH\Desktop\ANI PC\PhD Prepratery Work\GNN-Problems\sudoku-hard\sudoku-hard\train.csv')

Reading C:\Users\ANIMESH\Desktop\ANI PC\PhD Prepratery Work\GNN-Problems\sudoku-hard\sudoku-hard\train.csv...


In [49]:
data

[('...'),
 ...]

In [52]:
def encode(samples):
    def parse(x):
        return list(map(int, list(x)))

    encoded = [(parse(q), parse(a)) for q, a in samples]
    return encoded

data = encode(data)
data

[([0,
   0,
   3,
   0,
   0,
   0,
   0,
   7,
   0,
   0,
   0,
   0,
   9,
   0,
   0,
   4,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   3,
   7,
   0,
   0,
   2,
   4,
   0,
   7,
   0,
   0,
   0,
   6,
   0,
   0,
   0,
   9,
   0,
   0,
   0,
   6,
   7,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   8,
   3,
   0,
   0,
   0,
   0,
   0,
   5,
   0,
   0,
   0,
   7,
   0,
   8,
   0,
   2,
   0,
   0,
   0,
   0,
   0],
  [9,
   2,
   3,
   8,
   1,
   4,
   5,
   7,
   6,
   7,
   5,
   6,
   9,
   2,
   3,
   4,
   8,
   1,
   8,
   1,
   4,
   7,
   6,
   5,
   3,
   2,
   9,
   5,
   6,
   8,
   4,
   3,
   7,
   1,
   9,
   2,
   4,
   3,
   7,
   1,
   9,
   2,
   6,
   5,
   8,
   2,
   9,
   1,
   5,
   8,
   6,
   7,
   4,
   3,
   1,
   7,
   2,
   6,
   4,
   9,
   8,
   3,
   5,
   6,
   4,
   9,
   3,
   5,
   8,
   2,
   1,
   7,
   3,
   8,
   5,
   2,
   7,
   1,
   9,
   6,
   4]),
 ([0,
   5,
   0,
   7,
 

## Implementing RRN Class and layers

In [53]:
import torch
import dgl.function as fn
from torch import nn

In [ ]:
class RRNLayer(nn.Module):
    def __init__(self,msg_layer, node_update_function,edge_drop):
        super(RRNLayer,self).__init__()
        self.msg_layer = msg_layer
        self.node_update_function = node_update_function
        self.edge_dropout = nn.Dropout(edge_drop)


    def forward(self,g):
        g.apply_edges(self.get_msg)
        g.edata['e'] = self.edge_dropout(g.edata['e'])
        g.update_all(message_func=fn.copy_e('e','msg'),reduce_func=fn.sum('msg','m'))

        g.apply_nodes(self.node_update)
    
    def get_msg(self,edges):
        e = torch.cat